In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import db_engine
from sqlalchemy import text

print("\n" + "="*100)
print("ELITEX V7 - DATA QUALITY & COMPLETENESS ANALYSIS")
print("="*100)
print(f"Analysis Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*100 + "\n")

# Create output directory
OUTPUT_DIR = Path("Output")
OUTPUT_DIR.mkdir(exist_ok=True)

# Database engine
engine = db_engine.elite_engine




ELITEX V7 - DATA QUALITY & COMPLETENESS ANALYSIS
Analysis Started: 2025-10-16 11:43:35



## STEP 1: GET UNIQUE CLIENTS FROM CLIENT_CONTEXT (BASE TABLE)

In [6]:
print("STEP 1: Loading unique clients from core.client_context...")
print("-" * 100)

query_clients = """
SELECT * FROM core.client_context
WHERE client_id IS NOT NULL
"""

df_clients = pd.read_sql(query_clients, engine)
df_clients = df_clients.drop_duplicates(subset=['client_id'])
df_clients['client_id'] = df_clients['client_id'].str.upper()

total_clients = len(df_clients)
print(f"✓ Loaded {total_clients:,} unique clients")
print(f"  Total columns from client_context: {len(df_clients.columns)}")
print()


STEP 1: Loading unique clients from core.client_context...
----------------------------------------------------------------------------------------------------
  Unique clients: 51,179
  Columns from client_context: 35



## STEP 2: LOAD CLIENT_INVESTMENT DATA

In [5]:
print("STEP 2: Loading core.client_investment...")
print("-" * 100)

query_investment = """
SELECT * FROM core.client_investment
"""

try:
    df_investment = pd.read_sql(query_investment, engine)
    df_investment['client_id'] = df_investment['client_id'].str.upper()
    
    # Aggregate by client - for numeric columns sum, for others take first
    numeric_cols = df_investment.select_dtypes(include=['number']).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col != 'client_id']
    
    agg_dict = {col: 'sum' for col in numeric_cols}
    for col in df_investment.columns:
        if col not in numeric_cols and col != 'client_id':
            agg_dict[col] = 'first'
    
    df_investment_agg = df_investment.groupby('client_id').agg(agg_dict).reset_index()
    df_investment_agg.columns = ['client_id'] + [f'inv_{col}' for col in df_investment_agg.columns[1:]]
    
    print(f"✓ Loaded {len(df_investment):,} investment records with {len(df_investment.columns)} columns")
    print(f"  Unique clients with investments: {df_investment['client_id'].nunique():,}")
    print(f"  Aggregated to: {len(df_investment_agg)} clients with {len(df_investment_agg.columns)} columns")
except Exception as e:
    print(f"✗ Error loading client_investment: {e}")
    df_investment_agg = pd.DataFrame(columns=['client_id'])

print()


STEP 2: Loading core.client_investment...
----------------------------------------------------------------------------------------------------
✓ Loaded 1,955 investment records
  Unique clients with investments: 944
  Aggregated to client level: 944 rows
  client_id portfolio_id  total_cost_value_aed  total_market_value_aed  \
0   10GLPHG   1019461-41           3700000.000              3700000.00   
1   10GRRXX   1013322-31           5000000.000              5197900.00   
2   10PKFPQ   1047540-31             40478.113                41909.19   
3   10QAXPK   1008247-31            185812.700               137010.51   
4   10QHKPA   1006748-31             97864.257               149971.35   
5   10QHLHP   1006964-31              4080.703                 3085.32   
6   10QXGLF   1002195-31            517158.400                98011.80   
7   10QXRPX   1002342-31             92014.527               145161.74   
8   10RXAKP   1032874-31            487041.563               374783.92   
9   1

## STEP 3: LOAD CLIENT_PORTFOLIO DATA

In [7]:
print("STEP 3: Loading core.client_portfolio...")
print("-" * 100)

query_portfolio = """
SELECT * FROM core.client_portfolio
"""

try:
    df_portfolio = pd.read_sql(query_portfolio, engine)
    df_portfolio['client_id'] = df_portfolio['client_id'].str.upper()
    
    # Take most recent record per client (if date column exists, otherwise take first)
    if 'last_valuation_date' in df_portfolio.columns:
        df_portfolio_latest = df_portfolio.sort_values('last_valuation_date', ascending=False).groupby('client_id').first().reset_index()
    else:
        df_portfolio_latest = df_portfolio.groupby('client_id').first().reset_index()
    
    # Rename columns to avoid conflicts
    df_portfolio_latest.columns = ['client_id'] + [f'port_{col}' for col in df_portfolio_latest.columns[1:]]
    
    print(f"✓ Loaded {len(df_portfolio):,} portfolio records with {len(df_portfolio.columns)} columns")
    print(f"  Unique clients with portfolio: {df_portfolio['client_id'].nunique():,}")
    print(f"  Aggregated to: {len(df_portfolio_latest)} clients with {len(df_portfolio_latest.columns)} columns")
except Exception as e:
    print(f"✗ Error loading client_portfolio: {e}")
    df_portfolio_latest = pd.DataFrame(columns=['client_id'])

print()


STEP 3: Loading core.client_portfolio...
----------------------------------------------------------------------------------------------------
✓ Loaded 1,958 portfolio records
  Unique clients with portfolio: 1,956
  client_id last_valuation_date        aum  investible_cash deposits  \
0   10FGALK                None        0.0         30449.68     None   
1   10GGLFG                None        0.0        248006.80     None   
2   10GLFHG                None        0.0           152.39     None   
3   10GLPHG                None  3700000.0             0.33     None   
4   10GPKGQ                None        0.0        297681.30     None   

  asset_distribution  
0               None  
1               None  
2               None  
3               None  
4               None  


## STEP 4: LOAD PRODUCTBALANCE (using customer_number)

In [8]:
print("STEP 4: Loading core.productbalance (maps to customer_number)...")
print("-" * 100)

query_productbalance = """
SELECT * FROM core.productbalance
"""

try:
    df_productbalance = pd.read_sql(query_productbalance, engine)
    df_productbalance['customer_number'] = df_productbalance['customer_number'].str.upper()
    
    # Aggregate numeric columns by customer
    numeric_cols = df_productbalance.select_dtypes(include=['number']).columns.tolist()
    agg_dict = {col: 'sum' for col in numeric_cols}
    agg_dict['account_number'] = 'count'  # count accounts
    
    for col in df_productbalance.columns:
        if col not in numeric_cols and col != 'customer_number' and col != 'account_number':
            if col in agg_dict:
                continue
            agg_dict[col] = 'first'
    
    df_pb_agg = df_productbalance.groupby('customer_number').agg(agg_dict).reset_index()
    df_pb_agg.columns = ['client_id'] + [f'pb_{col}' for col in df_pb_agg.columns[1:]]
    
    print(f"✓ Loaded {len(df_productbalance):,} product balance records with {len(df_productbalance.columns)} columns")
    print(f"  Unique customers with products: {df_productbalance['customer_number'].nunique():,}")
    print(f"  Aggregated to: {len(df_pb_agg)} clients with {len(df_pb_agg.columns)} columns")
except Exception as e:
    print(f"✗ Error loading productbalance: {e}")
    df_pb_agg = pd.DataFrame(columns=['client_id'])

print()


STEP 4: Loading core.productbalance (maps to customer_number)...
----------------------------------------------------------------------------------------------------
✓ Loaded 113,028 product balance records
  Unique customers with products: 44,156
  client_id  total_outstanding_balance  product_count  \
0   10AAAKQ                     374.46              1   
1   10AAFXP                 -777953.17              3   
2   10AAGQH                 3695046.14              3   
3   10AAHAH                -1112816.39              3   
4   10AAKAX                   81077.37              1   

                        product_types  
0                    DEPOSIT PRODUCTS  
1  LENDING PRODUCTS, DEPOSIT PRODUCTS  
2  DEPOSIT PRODUCTS, LENDING PRODUCTS  
3  DEPOSIT PRODUCTS, LENDING PRODUCTS  
4                    DEPOSIT PRODUCTS  


## STEP 5: LOAD CLIENT_PROD_BALANCE_MONTHLY

In [9]:
print("STEP 5: Loading core.client_prod_balance_monthly...")
print("-" * 100)

query_monthly = """
SELECT * FROM core.client_prod_balance_monthly
"""

try:
    df_monthly = pd.read_sql(query_monthly, engine)
    df_monthly['client_id'] = df_monthly['client_id'].str.upper()
    
    # Get most recent month per client
    if 'year_cal' in df_monthly.columns and 'month_cal' in df_monthly.columns:
        df_monthly['year_cal'] = pd.to_numeric(df_monthly['year_cal'], errors='coerce')
        df_monthly['month_cal'] = pd.to_numeric(df_monthly['month_cal'], errors='coerce')
        df_monthly = df_monthly.sort_values(['year_cal', 'month_cal'], ascending=False)
    
    df_monthly_latest = df_monthly.groupby('client_id').first().reset_index()
    df_monthly_latest.columns = ['client_id'] + [f'monthly_{col}' for col in df_monthly_latest.columns[1:]]
    
    print(f"✓ Loaded {len(df_monthly):,} monthly balance records with {len(df_monthly.columns)} columns")
    print(f"  Unique clients with monthly data: {df_monthly['client_id'].nunique():,}")
    print(f"  Aggregated to: {len(df_monthly_latest)} clients with {len(df_monthly_latest.columns)} columns")
except Exception as e:
    print(f"✗ Error loading client_prod_balance_monthly: {e}")
    df_monthly_latest = pd.DataFrame(columns=['client_id'])

print()


STEP 5: Loading core.client_prod_balance_monthly...
----------------------------------------------------------------------------------------------------
✓ Loaded 262,212 monthly balance records
  Unique clients with monthly data: 44,536



In [20]:
print("STEP 6: Loading core.aecbalerts (maps to cif)...")
print("-" * 100)

query_aecb = """
SELECT * FROM core.aecbalerts
"""

try:
    df_aecb = pd.read_sql(query_aecb, engine)
    df_aecb['cif'] = df_aecb['cif'].str.upper()
    
    # Aggregate numeric columns
    numeric_cols = df_aecb.select_dtypes(include=['number']).columns.tolist()
    agg_dict = {col: 'sum' for col in numeric_cols}
    
    for col in df_aecb.columns:
        if col not in numeric_cols and col != 'cif':
            agg_dict[col] = 'first'
    
    df_aecb_agg = df_aecb.groupby('cif').agg(agg_dict).reset_index()
    df_aecb_agg.columns = ['client_id'] + [f'aecb_{col}' for col in df_aecb_agg.columns[1:]]
    
    print(f"✓ Loaded {len(df_aecb):,} AECB alert records with {len(df_aecb.columns)} columns")
    print(f"  Unique clients with AECB alerts: {df_aecb['cif'].nunique():,}")
    print(f"  Aggregated to: {len(df_aecb_agg)} clients with {len(df_aecb_agg.columns)} columns")
except Exception as e:
    print(f"✗ Error loading aecbalerts: {e}")
    df_aecb_agg = pd.DataFrame(columns=['client_id'])

print()


STEP 6: Loading core.aecbalerts (maps to cif)...
----------------------------------------------------------------------------------------------------
                               record_id              id      cif        cif2  \
0   a670eb70-4100-4113-8184-e237c7a2b97f  20240801003503  39QFGGR  80RHHHXGXH   
1   40f0e2e1-3ae4-4676-9534-18953584c39d  20240801003503  60PHGFH        None   
2   3e399b12-71ba-4f6a-8880-8f96701b46d9  20240801003503  12KGQAR     36KPKKH   
3   fa841029-becd-4bee-98e3-3dc4b70f897b  20240801003503  16QAFKF        None   
4   dc090681-4325-4f51-9d5e-d11113662e3f  20240801003503  44RGLKK        None   
..                                   ...             ...      ...         ...   
95  9f027bd1-a77a-4fa6-9261-c76cc0a69f9b  20240801003503  46QGXLA        None   
96  f0564d42-0e9a-43ea-b180-4523f463d07e  20240801003503  11LGFPX     38KKLXP   
97  f9f66785-84e7-4407-a0c6-9b27ad8dd990  20240801003503  41QAPRP        None   
98  f0b19dcf-6160-475c-b2e6-e6fd32292010

## STEP 6: LOAD AECB ALERTS (using cif)

In [12]:
print("STEP 6: Loading core.aecbalerts (maps to cif)...")
print("-" * 100)

query_aecb = """
SELECT * FROM core.aecbalerts
"""

try:
    df_aecb = pd.read_sql(query_aecb, engine)
    df_aecb['cif'] = df_aecb['cif'].str.upper()
    
    # Aggregate numeric columns
    numeric_cols = df_aecb.select_dtypes(include=['number']).columns.tolist()
    agg_dict = {col: 'sum' for col in numeric_cols}
    
    for col in df_aecb.columns:
        if col not in numeric_cols and col != 'cif':
            agg_dict[col] = 'first'
    
    df_aecb_agg = df_aecb.groupby('cif').agg(agg_dict).reset_index()
    df_aecb_agg.columns = ['client_id'] + [f'aecb_{col}' for col in df_aecb_agg.columns[1:]]
    
    print(f"✓ Loaded {len(df_aecb):,} AECB alert records with {len(df_aecb.columns)} columns")
    print(f"  Unique clients with AECB alerts: {df_aecb['cif'].nunique():,}")
    print(f"  Aggregated to: {len(df_aecb_agg)} clients with {len(df_aecb_agg.columns)} columns")
except Exception as e:
    print(f"✗ Error loading aecbalerts: {e}")
    df_aecb_agg = pd.DataFrame(columns=['client_id'])

print()


STEP 6: Loading core.aecbalerts (maps to cif)...
----------------------------------------------------------------------------------------------------
✓ Loaded 29,168 AECB alert records
  Unique clients with AECB alerts: 9,638
          client_id  aecb_total_amount  aecb_overdue_amount  aecb_alerts_count
0  00QQQQQQQQXGPLQQ                0.0                  0.0                  2
1           10AAFXP                0.0                  0.0                  1
2           10AALLX                0.0                  0.0                  4
3           10AGAFG                0.0                  0.0                  2
4           10AGAHP                0.0                 34.0                  4


## STEP 7: LOAD DEBIT TRANSACTIONS (using cif2)

In [ ]:
print("STEP 7: Skipping core.clienttransactiondebit (column mapping issues)...")
print("-" * 100)
df_debit_agg = pd.DataFrame(columns=['client_id'])
print("✓ Skipped")
print()


## STEP 8: LOAD CREDIT TRANSACTIONS (using cif2)

In [ ]:
print("STEP 8: Skipping core.clienttransactioncredit (column mapping issues)...")
print("-" * 100)
df_credit_agg = pd.DataFrame(columns=['client_id'])
print("✓ Skipped")
print()


## STEP 9: LOAD BANCASSURANCE DATA

In [ ]:
print("STEP 9: Loading core.bancaclientproduct...")
print("-" * 100)

query_banca = """
SELECT * FROM core.bancaclientproduct
"""

try:
    df_banca = pd.read_sql(query_banca, engine)
    df_banca['client_id'] = df_banca['client_id'].str.upper()
    
    # Aggregate by client
    numeric_cols = df_banca.select_dtypes(include=['number']).columns.tolist()
    agg_dict = {col: 'sum' for col in numeric_cols}
    agg_dict['policy_number'] = 'count'  # count policies
    
    for col in df_banca.columns:
        if col not in numeric_cols and col != 'client_id' and col != 'policy_number':
            if col in agg_dict:
                continue
            agg_dict[col] = 'first'
    
    df_banca_agg = df_banca.groupby('client_id').agg(agg_dict).reset_index()
    df_banca_agg.columns = ['client_id'] + [f'banca_{col}' for col in df_banca_agg.columns[1:]]
    
    print(f"✓ Loaded {len(df_banca):,} bancassurance records with {len(df_banca.columns)} columns")
    print(f"  Unique clients with bancassurance: {df_banca['client_id'].nunique():,}")
    print(f"  Aggregated to: {len(df_banca_agg)} clients with {len(df_banca_agg.columns)} columns")
except Exception as e:
    print(f"✗ Error loading bancaclientproduct: {e}")
    df_banca_agg = pd.DataFrame(columns=['client_id'])

print()


## STEP 10: LOAD UPSELL OPPORTUNITIES

In [ ]:
print("STEP 1: Loading unique clients from core.client_context...")
print("-" * 100)

query_clients = """
SELECT * FROM core.client_context
WHERE client_id IS NOT NULL
"""

df_clients = pd.read_sql(query_clients, engine)
df_clients = df_clients.drop_duplicates(subset=['client_id'])
df_clients['client_id'] = df_clients['client_id'].str.upper()

total_clients = len(df_clients)
print(f"✓ Loaded {total_clients:,} unique clients")
print(f"  Total columns from client_context: {len(df_clients.columns)}")
print()


## STEP 11: LOAD USER JOIN CLIENT CONTEXT (RM MAPPING)

In [ ]:
print("STEP 1: Loading unique clients from core.client_context...")
print("-" * 100)

query_clients = """
SELECT * FROM core.client_context
WHERE client_id IS NOT NULL
"""

df_clients = pd.read_sql(query_clients, engine)
df_clients = df_clients.drop_duplicates(subset=['client_id'])
df_clients['client_id'] = df_clients['client_id'].str.upper()

total_clients = len(df_clients)
print(f"✓ Loaded {total_clients:,} unique clients")
print(f"  Total columns from client_context: {len(df_clients.columns)}")
print()


## STEP 12: MERGE ALL DATA USING LEFT JOIN ON CLIENT_ID

In [ ]:
print("\n" + "="*100)
print("STEP 12: MERGING ALL DATA TABLES")
print("="*100)

# Start with base clients table
df_merged = df_clients.copy()
print(f"Starting with base: {len(df_merged):,} clients, {len(df_merged.columns)} columns")

# Merge investment data
if not df_investment_agg.empty:
    df_merged = df_merged.merge(df_investment_agg, on='client_id', how='left')
    print(f"After merging investment: {len(df_merged.columns)} columns")

# Merge portfolio data
if not df_portfolio_latest.empty:
    df_merged = df_merged.merge(df_portfolio_latest, on='client_id', how='left')
    print(f"After merging portfolio: {len(df_merged.columns)} columns")

# Merge product balance
if not df_pb_agg.empty:
    df_merged = df_merged.merge(df_pb_agg, on='client_id', how='left')
    print(f"After merging product balance: {len(df_merged.columns)} columns")

# Merge monthly balance
if not df_monthly_latest.empty:
    df_merged = df_merged.merge(df_monthly_latest, on='client_id', how='left')
    print(f"After merging monthly balance: {len(df_merged.columns)} columns")

# Merge AECB alerts
if not df_aecb_agg.empty:
    df_merged = df_merged.merge(df_aecb_agg, on='client_id', how='left')
    print(f"After merging AECB alerts: {len(df_merged.columns)} columns")

# Merge debit transactions
if not df_debit_agg.empty:
    df_merged = df_merged.merge(df_debit_agg, on='client_id', how='left')
    print(f"After merging debit transactions: {len(df_merged.columns)} columns")

# Merge credit transactions
if not df_credit_agg.empty:
    df_merged = df_merged.merge(df_credit_agg, on='client_id', how='left')
    print(f"After merging credit transactions: {len(df_merged.columns)} columns")

# Merge bancassurance
if not df_banca_agg.empty:
    df_merged = df_merged.merge(df_banca_agg, on='client_id', how='left')
    print(f"After merging bancassurance: {len(df_merged.columns)} columns")

# Merge upsell opportunities
if not df_upsell_agg.empty:
    df_merged = df_merged.merge(df_upsell_agg, on='client_id', how='left')
    print(f"After merging upsell opportunities: {len(df_merged.columns)} columns")

# Merge RM mapping
if not df_rm_unique.empty:
    df_merged = df_merged.merge(df_rm_unique, on='client_id', how='left')
    print(f"After merging RM mapping: {len(df_merged.columns)} columns")

print(f"\n✓ Final merged dataset: {len(df_merged):,} clients, {len(df_merged.columns)} total columns")
print()



## STEP 13: ANALYZE DATA COVERAGE AND MISSING VALUES

In [ ]:
print("\n" + "="*100)
print("STEP 13: ANALYZING DATA COVERAGE AND MISSING VALUES")
print("="*100)

# Calculate missing values per column
missing_analysis = []

for col in df_merged.columns:
    total_count = len(df_merged)
    non_null_count = df_merged[col].notna().sum()
    null_count = total_count - non_null_count
    coverage_pct = (non_null_count / total_count * 100) if total_count > 0 else 0
    
    missing_analysis.append({
        'column_name': col,
        'total_clients': total_count,
        'clients_with_data': non_null_count,
        'clients_missing_data': null_count,
        'coverage_pct': round(coverage_pct, 2),
        'missing_pct': round(100 - coverage_pct, 2)
    })

df_coverage = pd.DataFrame(missing_analysis)
df_coverage = df_coverage.sort_values('coverage_pct', ascending=True)

print(f"✓ Analyzed {len(df_coverage)} columns")
print(f"  Columns with 100% coverage: {len(df_coverage[df_coverage['coverage_pct'] == 100])}")
print(f"  Columns with <50% coverage: {len(df_coverage[df_coverage['coverage_pct'] < 50])}")
print()



## STEP 14: CATEGORIZE COLUMNS BY SOURCE TABLE

In [ ]:
print("STEP 14: Categorizing columns by source table...")
print("-" * 100)

# Map columns to source tables
column_source_map = []

# Client context columns (base table)
for col in df_clients.columns:
    column_source_map.append({
        'column_name': col,
        'source_table': 'core.client_context',
        'is_base_table': True
    })

# Investment columns
for col in df_investment_agg.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.client_investment',
            'is_base_table': False
        })

# Portfolio columns
for col in df_portfolio_latest.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.client_portfolio',
            'is_base_table': False
        })

# Product balance columns
for col in df_pb_agg.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.productbalance',
            'is_base_table': False
        })

# Monthly balance columns
for col in df_monthly_latest.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.client_prod_balance_monthly',
            'is_base_table': False
        })

# AECB columns
for col in df_aecb_agg.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.aecbalerts',
            'is_base_table': False
        })

# Debit transaction columns
for col in df_debit_agg.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.clienttransactiondebit',
            'is_base_table': False
        })

# Credit transaction columns
for col in df_credit_agg.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.clienttransactioncredit',
            'is_base_table': False
        })

# Bancassurance columns
for col in df_banca_agg.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.bancaclientproduct',
            'is_base_table': False
        })

# Upsell columns
for col in df_upsell_agg.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'app.upsellopportunity',
            'is_base_table': False
        })

# RM mapping columns
for col in df_rm_unique.columns:
    if col != 'client_id':
        column_source_map.append({
            'column_name': col,
            'source_table': 'core.user_join_client_context',
            'is_base_table': False
        })

df_column_sources = pd.DataFrame(column_source_map)

# Merge with coverage analysis
df_coverage_detailed = df_coverage.merge(df_column_sources, on='column_name', how='left')
df_coverage_detailed['source_table'] = df_coverage_detailed['source_table'].fillna('Unknown')

print(f"✓ Categorized {len(df_coverage_detailed)} columns by source table")
print()



## STEP 15: GENERATE SUMMARY STATISTICS

In [ ]:
print("STEP 15: Generating summary statistics...")
print("-" * 100)

summary_stats = {
    'total_unique_clients': total_clients,
    'total_columns_in_merged_data': len(df_merged.columns),
    'columns_from_client_context': len([c for c in df_coverage_detailed['source_table'] if c == 'core.client_context']),
    'columns_with_100pct_coverage': len(df_coverage[df_coverage['coverage_pct'] == 100]),
    'columns_with_50_to_100pct_coverage': len(df_coverage[(df_coverage['coverage_pct'] >= 50) & (df_coverage['coverage_pct'] < 100)]),
    'columns_with_less_than_50pct_coverage': len(df_coverage[df_coverage['coverage_pct'] < 50]),
    'average_coverage_pct': round(df_coverage['coverage_pct'].mean(), 2),
    'median_coverage_pct': round(df_coverage['coverage_pct'].median(), 2),
}

print("Summary Statistics:")
print("-" * 100)
for key, value in summary_stats.items():
    print(f"  {key}: {value}")
print()



## STEP 16: IDENTIFY KEY MISSING DATA ISSUES

In [ ]:
print("STEP 16: Identifying key missing data issues...")
print("-" * 100)

# Find columns with significant missing data (>50% missing)
high_missing = df_coverage_detailed[df_coverage_detailed['missing_pct'] > 50].sort_values('missing_pct', ascending=False)

print(f"Columns with >50% missing data: {len(high_missing)}")
if len(high_missing) > 0:
    print("\nTop 10 columns with highest missing data:")
    print("-" * 100)
    for idx, row in high_missing.head(10).iterrows():
        print(f"  {row['column_name']:<40} {row['source_table']:<40} {row['missing_pct']:.1f}% missing")
print()



## STEP 17: EXPORT ALL RESULTS TO EXCEL

In [ ]:
print("\n" + "="*100)
print("STEP 17: EXPORTING RESULTS TO EXCEL")
print("="*100)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = OUTPUT_DIR / f"EliteX_Data_Quality_Report_{timestamp}.xlsx"

print("Creating comprehensive Excel report with all data tables...")
print("-" * 100)

# Create table summary before writing
table_summary = df_coverage_detailed.groupby('source_table').agg({
    'column_name': 'count',
    'coverage_pct': 'mean'
}).reset_index()
table_summary.columns = ['source_table', 'column_count', 'avg_coverage_pct']
table_summary = table_summary.sort_values('avg_coverage_pct', ascending=False)

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    # Sheet 1: Executive Summary
    df_summary = pd.DataFrame([summary_stats]).T
    df_summary.columns = ['Value']
    df_summary.index.name = 'Metric'
    df_summary.to_excel(writer, sheet_name='Executive Summary')
    print("✓ Sheet 1: Executive Summary")
    
    # Sheet 2: Merged Data (first 10,000 rows)
    df_merged.head(10000).to_excel(writer, sheet_name='Merged Data (Sample)', index=False)
    print(f"✓ Sheet 2: Merged Data (Sample - first {min(10000, len(df_merged))} rows)")
    
    # Sheet 3: Column Coverage Analysis
    df_coverage_detailed.to_excel(writer, sheet_name='Column Coverage', index=False)
    print(f"✓ Sheet 3: Column Coverage ({len(df_coverage_detailed)} columns)")
    
    # Sheet 4: High Missing Data
    if not high_missing.empty:
        high_missing.to_excel(writer, sheet_name='High Missing Data', index=False)
        print(f"✓ Sheet 4: High Missing Data ({len(high_missing)} columns)")
    
    # Sheet 5: Coverage by Table
    table_summary.to_excel(writer, sheet_name='Coverage by Table', index=False)
    print(f"✓ Sheet 5: Coverage by Table ({len(table_summary)} tables)")
    
    # NEW: Export all raw tables with ALL columns
    print("\nExporting raw data tables...")
    print("-" * 100)
    
    # Client Context (full data)
    if not df_clients.empty:
        df_clients.to_excel(writer, sheet_name='RAW_ClientContext', index=False)
        print(f"✓ RAW_ClientContext: {len(df_clients):,} rows, {len(df_clients.columns)} columns")
    
    # Investment (full raw data, not aggregated)
    if 'df_investment' in dir() and not df_investment.empty:
        df_investment.to_excel(writer, sheet_name='RAW_Investment', index=False)
        print(f"✓ RAW_Investment: {len(df_investment):,} rows, {len(df_investment.columns)} columns")
    
    # Portfolio (full raw data)
    if 'df_portfolio' in dir() and not df_portfolio.empty:
        df_portfolio.to_excel(writer, sheet_name='RAW_Portfolio', index=False)
        print(f"✓ RAW_Portfolio: {len(df_portfolio):,} rows, {len(df_portfolio.columns)} columns")
    
    # Product Balance (full raw data)
    if 'df_productbalance' in dir() and not df_productbalance.empty:
        # Limit to 100k rows for Excel size
        df_pb_export = df_productbalance.head(100000)
        df_pb_export.to_excel(writer, sheet_name='RAW_ProductBalance', index=False)
        print(f"✓ RAW_ProductBalance: {len(df_pb_export):,} rows (limited), {len(df_pb_export.columns)} columns")
    
    # Monthly Balance (full raw data)
    if 'df_monthly' in dir() and not df_monthly.empty:
        # Limit to 50k rows
        df_monthly_export = df_monthly.head(50000)
        df_monthly_export.to_excel(writer, sheet_name='RAW_MonthlyBalance', index=False)
        print(f"✓ RAW_MonthlyBalance: {len(df_monthly_export):,} rows (limited), {len(df_monthly_export.columns)} columns")
    
    # AECB Alerts (full raw data)
    if 'df_aecb' in dir() and not df_aecb.empty:
        df_aecb.to_excel(writer, sheet_name='RAW_AECB_Alerts', index=False)
        print(f"✓ RAW_AECB_Alerts: {len(df_aecb):,} rows, {len(df_aecb.columns)} columns")
    
    # Debit Transactions (sample due to size)
    if 'df_debit' in dir() and not df_debit.empty:
        df_debit_export = df_debit.head(50000)
        df_debit_export.to_excel(writer, sheet_name='RAW_DebitTxn', index=False)
        print(f"✓ RAW_DebitTxn: {len(df_debit_export):,} rows (limited), {len(df_debit_export.columns)} columns")
    
    # Credit Transactions (sample due to size)
    if 'df_credit' in dir() and not df_credit.empty:
        df_credit_export = df_credit.head(50000)
        df_credit_export.to_excel(writer, sheet_name='RAW_CreditTxn', index=False)
        print(f"✓ RAW_CreditTxn: {len(df_credit_export):,} rows (limited), {len(df_credit_export.columns)} columns")
    
    # Bancassurance (full raw data)
    if 'df_banca' in dir() and not df_banca.empty:
        df_banca.to_excel(writer, sheet_name='RAW_Bancassurance', index=False)
        print(f"✓ RAW_Bancassurance: {len(df_banca):,} rows, {len(df_banca.columns)} columns")
    
    # Upsell (full raw data)
    if 'df_upsell' in dir() and not df_upsell.empty:
        df_upsell.to_excel(writer, sheet_name='RAW_Upsell', index=False)
        print(f"✓ RAW_Upsell: {len(df_upsell):,} rows, {len(df_upsell.columns)} columns")
    
    # RM Mapping (full raw data)
    if 'df_rm' in dir() and not df_rm.empty:
        df_rm.to_excel(writer, sheet_name='RAW_RM_Mapping', index=False)
        print(f"✓ RAW_RM_Mapping: {len(df_rm):,} rows, {len(df_rm.columns)} columns")

print("\n" + "="*100)
print(f"✓ Excel report saved: {output_file}")
print("="*100)
print()


## STEP 18: PRINT FINAL SUMMARY TO CONSOLE

In [ ]:
print("\n" + "="*100)
print("ANALYSIS COMPLETE - FINAL SUMMARY")
print("="*100)
print(f"\nTotal Unique Clients: {total_clients:,}")
print(f"Total Columns in Merged Dataset: {len(df_merged.columns)}")
print(f"Average Column Coverage: {summary_stats['average_coverage_pct']}%")
print(f"Columns with <50% Coverage: {summary_stats['columns_with_less_than_50pct_coverage']}")

print(f"\n✓ Full report saved to: {output_file}")
print("="*100 + "\n")

print("Analysis finished successfully!")
print(f"Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
